## Fine tuning the model using the criteries created by the model.


## Outline

 * Direct API calls to OpenAI

   * Prompts
   

## Get your [OpenAI API Key](https://platform.openai.com/account/api-keys)

In [1]:
#!pip install openai==0.28.1

In [8]:
#!pip install openai --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 1.1 MB/s eta 0:00:000:00:01m eta 0:00:01
  Attempting uninstall: openai
    Found existing installation: openai 0.28.1
    Uninstalling openai-0.28.1:
      Successfully uninstalled openai-0.28.1


In [1]:
import os
import openai
from langchain.chat_models import ChatOpenAI, ChatGooglePalm

from pprint import pprint
from utils import OPENAI_API_KEY

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY 

openai.api_key = os.environ['OPENAI_API_KEY']

In [155]:
openai.models.list().data

[Model(id='text-search-babbage-doc-001', created=1651172509, object='model', owned_by='openai-dev'),
 Model(id='gpt-3.5-turbo-16k', created=1683758102, object='model', owned_by='openai-internal'),
 Model(id='curie-search-query', created=1651172509, object='model', owned_by='openai-dev'),
 Model(id='text-davinci-003', created=1669599635, object='model', owned_by='openai-internal'),
 Model(id='text-search-babbage-query-001', created=1651172509, object='model', owned_by='openai-dev'),
 Model(id='babbage', created=1649358449, object='model', owned_by='openai'),
 Model(id='babbage-search-query', created=1651172509, object='model', owned_by='openai-dev'),
 Model(id='whisper-1-1p', created=1700094243, object='model', owned_by='system'),
 Model(id='text-babbage-001', created=1649364043, object='model', owned_by='openai'),
 Model(id='text-similarity-davinci-001', created=1651172505, object='model', owned_by='openai-dev'),
 Model(id='davinci-similarity', created=1651172509, object='model', owned

In [156]:

llm_model = "gpt-3.5-turbo-1106"
llm_model = 'gpt-4-0613'

## Chat API : OpenAI

Let's start with a direct API calls to OpenAI.

In [157]:
def get_completion(prompt, model=llm_model):
    
    messages = [{"role": "system", "content": "You are a helpful teacher assistant"},
                {"role": "user", "content": prompt}]
    try:
        from openai import ChatCompletion
        
        response = ChatCompletion.create(
                   model=model,
                   messages=messages,
                   temperature=0, 
                    )
        
        return response#.choices[0].message["content"]
    
    except:
        from openai import OpenAI
        
        client = OpenAI(api_key=OPENAI_API_KEY )
        response = client.chat.completions.create(
                          model=model,
                   messages=messages,
                   temperature=0 
                   )
        return response.choices[0].message.content  
    


In [158]:
chat = ChatOpenAI(temperature=0.0, model=llm_model, verbose=True)

In [159]:
# Check connection
get_completion("What is 1+1?")#, chat("What is 1+1?")

'1+1 equals 2.'

In [32]:
rubrics = """ Organization, Content, Grammar, Vocabulary, Overall impression """


In [33]:
# Criteries were created by the model on the previous step.
criteries = {
  "Organization": {
    "1": "Essay lacks clear structure and coherence. Ideas are disorganized and difficult to follow.",
    "2": "Essay has some organization, but transitions between ideas are weak. Some parts may be unclear or confusing.",
    "3": "Essay has a logical structure with clear transitions between ideas. Most parts are clear and easy to follow.",
    "4": "Essay is well-organized with strong transitions between ideas. All parts are clear and coherent.",
    "5": "Essay has a highly effective organization with seamless transitions between ideas. All parts are clear, coherent, and engaging.",
    "6": "Essay demonstrates exceptional organization. Ideas flow logically and coherently. The structure enhances the overall impact of the essay."
  },
  "Content": {
    "1": "Essay lacks relevant and meaningful content. Ideas are superficial and lack depth.",
    "2": "Essay has some relevant content, but it lacks depth and originality. Some ideas may be repetitive or irrelevant.",
    "3": "Essay contains mostly relevant content with some depth and originality. Ideas are adequately developed.",
    "4": "Essay demonstrates substantial and insightful content. Ideas are well-developed and show originality.",
    "5": "Essay exhibits highly relevant and thought-provoking content. Ideas are exceptionally developed and demonstrate originality.",
    "6": "Essay showcases exceptional and profound content. Ideas are exceptionally developed, insightful, and demonstrate exceptional originality."
  },
  "Grammar": {
    "1": "Essay has numerous grammatical errors that impede understanding. Sentence structure and word choice are consistently incorrect.",
    "2": "Essay has several grammatical errors that occasionally impede understanding. Sentence structure and word choice are often incorrect.",
    "3": "Essay has some grammatical errors, but they do not significantly impede understanding. Sentence structure and word choice are generally correct.",
    "4": "Essay has few grammatical errors. Sentence structure and word choice are mostly correct.",
    "5": "Essay has virtually no grammatical errors. Sentence structure and word choice are consistently correct.",
    "6": "Essay is virtually flawless in terms of grammar. Sentence structure and word choice are impeccable."
  },
  "Vocabulary": {
    "1": "Essay has limited and inappropriate vocabulary. Word choice is often incorrect or unclear.",
    "2": "Essay has some variety in vocabulary, but it lacks precision and sophistication. Word choice is occasionally incorrect or unclear.",
    "3": "Essay has a satisfactory range of vocabulary with some precision and sophistication. Word choice is generally correct and clear.",
    "4": "Essay demonstrates a good range of vocabulary with precision and sophistication. Word choice is mostly correct and clear.",
    "5": "Essay exhibits a wide range of vocabulary with precision and sophistication. Word choice is consistently correct and clear.",
    "6": "Essay showcases an exceptional range of vocabulary with precision and sophistication. Word choice is impeccable."
  },
  "Overall impression": {
    "1": "Essay leaves a negative overall impression. It lacks coherence, depth, and clarity.",
    "2": "Essay leaves a somewhat negative overall impression. It lacks coherence or depth in some areas.",
    "3": "Essay leaves a neutral overall impression. It is generally coherent and adequately developed.",
    "4": "Essay leaves a positive overall impression. It is well-organized, coherent, and developed.",
    "5": "Essay leaves a highly positive overall impression. It is exceptionally well-organized, coherent, and developed.",
    "6": "Essay leaves an outstanding overall impression. It is exceptionally well-organized, coherent, developed, and engaging."
  }
}


In [34]:
import json
import numpy as np

def get_feedback(score, criteries=criteries):
    """
    Get feedback as a text using the score and criteries of scores
    """
    return json.dumps({k: {score :v[str(score)]} for k,v in criteries.items()}, indent=4)

def get_random_scores(score, n):
    """
    Get feedbacks length n that give the score as a mean
    """
    
    l_score = int(score)
    h_score = l_score
    if score % 1 != 0:
        h_score = l_score+1
    _scores = np.random.choice([l_score, h_score], size=n )
    
    #score_l = score
    if (score *10) % 2 != 0:
        score = round(score-0.1, 1)
    
    while round(_scores.mean(), 1) != score:
        #print(_scores.mean(), score)
        _scores = np.random.choice([l_score, h_score], size=n)
    return _scores


def get_feedback(score, criteries=criteries):
    """
    Get feedback using the score for each criteria.
    """
    scores = get_random_scores(score, len(criteries))
    d_scores = {k:str(v) for k,v in zip(criteries.keys(), scores)}
    return json.dumps({k: {d_scores[k] :v[str(d_scores[k])]} for k,v in criteries.items()}, indent=4)

In [35]:
get_feedback(5.6)

'{\n    "Organization": {\n        "5": "Essay has a highly effective organization with seamless transitions between ideas. All parts are clear, coherent, and engaging."\n    },\n    "Content": {\n        "6": "Essay showcases exceptional and profound content. Ideas are exceptionally developed, insightful, and demonstrate exceptional originality."\n    },\n    "Grammar": {\n        "6": "Essay is virtually flawless in terms of grammar. Sentence structure and word choice are impeccable."\n    },\n    "Vocabulary": {\n        "6": "Essay showcases an exceptional range of vocabulary with precision and sophistication. Word choice is impeccable."\n    },\n    "Overall impression": {\n        "5": "Essay leaves a highly positive overall impression. It is exceptionally well-organized, coherent, and developed."\n    }\n}'

### Example grading
##### https://admitreport.com/blog/college-essay-examples

In [36]:
# 1: The Gospel of Steve,  Grade:A+

essay_A1 = """
In sophomore year, I struggled with depression. I felt like 
I was constantly battling against the darkness that seemed to be closing in on me. Until, that is, 
I found solace in the teachings of Steve Irwin.

When I first discovered Steve Irwin and his show "The Crocodile Hunter," 
I was captivated by his passion for wildlife. He was fearless, jumping into danger without hesitation to save an animal in need. 
But it was more than just his bravery that inspired me; it was his infectious energy and love for life. 
Watching him on TV, I couldn't help but feel a little bit better about my own struggles.

But it wasn't until I read his biography that I truly felt the impact Steve had on my life. 
In the book, he talked openly about his own struggles with depression. He talked about the dark moments in his life, 
when he felt like he was drowning in despair. But he also talked about how he fought back against the darkness, 
how he refused to let it consume him, and how he turned his depression into a career that allowed him 
to follow his biggest passions.

Reading Steve's words, I felt like he was speaking directly to me. 
I wasn't alone in my struggles if someone as brave and fearless as Steve had faced similar challenges. 
And that gave me the courage to keep going. I started visiting a therapist, exercising regularly, and practicing 
mindfulness meditation. Day by day, I lifted myself out of my depression–all with a healthy dose of 
“Crocodile Hunter” each evening after I finished my homework.

One of the things that I admired most about Steve was his ability to find joy and 
laughter in the most unlikely places. He was always cracking jokes, even in the face of danger. 
He taught me that laughter and humor can be a powerful tool in the fight against depression. 
I went looking for the humor in my own struggles. I started learning about how stand-up comedy works, 
and wrote my own five-minute skit finding the humor and silver lining in my depression. I wasn’t a great comic, 
let me tell you. But being able to channel my experience into something positive—something that helped others 
laugh—was extremely gratifying to me.

Depression is a bizarre thing. One day, you’re besieged by it from every side and it looks like there’s no way out. 
Then, two months later, if you’re diligent, you look around the world and wonder what you ever had to be upset about.0
You find goodness and light in the things around you—your friends, your family, your habits, and your hobbies.
These forces act as buttresses to keep you standing up and moving forward.

As silly as it may sound, I credit Steve Irwin with that first buttress. 
His experience and outlook on life gave me the push I needed to cultivate bravery and resilience 
in the face of my struggle with mental health. My eternal goal is now to practice the gospel of 
Steve—to always pass along humor, passion, and encouragement to others, especially to those who seem down and out.
Thank you, Steve."""

In [37]:
#2: The Embroidery Scientist, Grade:A

essay_A2 = """
I stretch the thin fabric over my hoop and pull it tight, wedging the nested rings between my legs to secure them shut with my other hand.

Next I get out the thread. Each color is wound tightly around a paper spool and stored in a container whose original purpose was to store fishing tackle.

I look at the pre-printed design on the fabric and decide what colors to select. Orange, red, pink, yellow–this design will be as bright and happy as I can make it.

Embroidery is where the STEM and creative parts of my identity converge. My STEM side is calculated. She meticulously plans the designs, mocks them up in photoshop, and painstakingly transfers them onto the fabric. She organizes each thread color by its place in ROYGBIV and cuts every piece to an identical length of 18”. Her favorite stitch is the French Knot, with its methodical “one, two” wrap sequence. For her, art is about precision.

My creative side, on the other hand, is messy. She throws thread scraps on the floor without hesitation, and she haphazardly adds design elements in pen. She does a Lazy Daisy stitch very lazily while adding an indescribable flourish to a simple backstitch. Her methods are indeed madness: she’ll border a design with glitter glue, hang a finished project upside down, or stitch a big red X over a perfectly good embroidery. For her, art is about meaning.

While these two sides of myself may seem at odds, they actually complement each other perfectly. At least, that’s what 3,000 of my Etsy customers think. From three-inch hoops to massive wall hangings, my Etsy shop is a compilation of the best embroidery I’ve ever done. My precision and meaning have earned me hundreds of five-star reviews from customers whose lives I’ve impacted with my art. And none of that art would have been possible without STEM me and creative me.

My STEM and creative side complement each other in more than my embroidery life too. What began as a creative side hustle has actually made me a better scientist.

Before I started embroidering, I approached the lab bench with an eye like a ruler. Poured a millimeter too much liquid? Better get a pipette. Went a degree over boiling? Time to start over. My lab reports demonstrated my knowledge, skill, and care, but they didn’t show any innovation or ingenuity. My precision led me to be a good scientist but not an exceptional one.

I realized that to be exceptional, I needed to think like a real scientist. While scientists are careful and precise, they are also interrogators. They constantly question the world around them, identifying previously unseen problems and finding creative solutions. To become the scientist I wanted to be, I needed to allow myself to be more creative.

When I had this realization, I had just begun my embroidery business. I didn’t understand that my creativity could also be so useful in the lab. I set out on a new path to use more creativity in the pursuit of science.

To inspire myself, I brought an embroidery project to the lab. On it, I stitched a compound microscope and a quote from one of my favorite scientists, Marie Curie. It reads, “I am among those who think that science has great beauty.”

In the lab now, I’m not afraid to take risks and try new things. When I boil my mixture too long, I still start over. But occasionally, when my teacher permits, I do a second experiment on the rejected liquid just to see what will happen. Sometimes nothing happens. Sometimes it results in utter failure. But other times, my mistakes create blue, green, and purple mixtures, mixtures that bubble and burst and fizz. All of these experiments are stitches in my quest to become a cancer researcher. They are messy, but they are beautiful.
"""

In [38]:
#4: Poetry Slam, Grade:A

essay_A4 = """
When I first met Simon, he was neither speaking nor singing. He was doing something in between. With words that flowed together like an ancient tributary, he spoke music. His hands grasping a microphone, he swayed slowly from side to side. He was a poet. But unlike that of Yeats or Dickenson, Simon’s poetry wasn’t meant to be read on a page—it was meant to be experienced like an aural work of art. And I had never experienced anything more beautiful. Disheartened, I realized that my words would never sound like Simon’s.

I sat in my on-deck seat. Forgetting that I was up next, I admired his craft. The crescendos and decrescendos that mirrored his pacing, the quick staccatos that punctuated each stanza, the rhymes so subtle they almost disappeared—every second of his spoken word pulled me further from reality. I listened to his words like a devout in church. Closing my eyes, I joined my hands together to count the syllables. From the outside, it probably looked like I was praying. And maybe I was. When Simon’s poem ended, the audience, though betrayed by the silence, erupted into applause.

It was my turn. I had spent an entire year perfecting my poem. My sister had grown accustomed to kicking me under the dinner table when someone asked me a question. She knew that my mind was in my beloved poetry notebook, mentally analyzing my latest draft. I’ve never been one for living in the moment. My report cards usually feature comments like, “She’s a good student but has trouble paying attention.” I’m always the first one out in dodgeball because my mind is completely absent from the school gym. But what seems like inattention to my teachers is actually a kind of profound focus.

When writing slam poetry, I become completely consumed. I like to start with the words. The rhythm and intonation come with time. For me, it’s about translating a feeling into language. It’s no easy task, but it feels like an obligation. Once the words come into being, they’re like a twister in my mind. They spin and spin, destroying every other thought in their path. I can’t focus on anything else because, in the aftermath of a twister, nothing else exists.

And there on the stage, nothing else existed besides me and my poem. I spoke it into existence. Like Simon, I wrapped my hands around the microphone, willing my poem to be heard. The twister exited my mind and entered the world.

A few weeks ago, I watched the recording of my first poetry slam, that slam two years ago when I saw Simon perform for the first time. I saw myself climb on stage from the dark abyss of the audience. I looked small, all alone on that big stage. My voice shook as I began. But soon, my poem rendered the stage smaller and smaller. I filled the darkness with words.

As I watched myself on my computer, I thought about how I felt that day, awe-struck in the audience by Simon’s work. I felt like I’d never be able to sound like him. And I was right. My poem didn’t sound like Simon’s, and none of my poems ever would. But in this moment, I realized that they were just as beautiful. My words sounded like me.
"""

In [39]:
#5: The Stop Sign, Grade: A+

essay_A5 = """
While some high schoolers get in trouble for skipping class, I get in trouble for arguing with my local government officials on Twitter. But when lives are at stake, I can take the heat.

I live at the intersection of 33rd and Spruce. The intersection itself sits between a large bend and a bundle of white oak trees—a recipe for obstructed views. Drivers careen around the corner, Indy 500-style, and are abruptly met with oncoming traffic. Neither can see the other through the oaks. What is otherwise a beautiful intersection makes for awfully dangerous driving conditions.

Living by this intersection my whole life, I’ve heard countless crashes and collisions. The screeching tires and cacophony of crushing car parts is seared in my mind. As neighbors, we are often the first on the scene. Cell phone in hand, I’ve run out to help several motorists who didn’t know what was coming. After the most recent crash, where a car flipped into the ditch, I knew that something had to change. We needed a stop sign.

I began with a google search, which led me to my local Stop Sign Request Form. According to the form, a government official would reach out to me. If they deemed it appropriate, we’d work together to assess whether the intersection qualified for a stop sign.

Their response took months. While I waited, I began collecting evidence on my own. After noticing that the security camera on my house pointed toward the intersection, I decided to put the skills I’d been developing in AP Computer Science to work. I wrote a simple code that tabulated the number of cars that passed through the intersection each day. Briefly reviewing the footage each night also helped me determine how many cars were likely going over the posted speed limit of forty miles per hour. Alongside these statistics, I went back into our cloud history to find footage of the crashes that had occurred.

When I finally heard back from the city, I was ready to make my case. My confidence deflated as soon as I opened the email: Thank you for filling out a Stop Sign Request Form, the email read. At this time, we do not have reason to believe that the intersection of 33rd Street and Spruce Street meets the criteria for a two-way stop sign. The city had disagreed with my recommendation and denied my request.

I took a moment to collect myself. How could the city not care about the safety of its citizens? Were human lives not worth looking into a simple stop sign? I took to Twitter, posting statistics from my research, photos of the obstructed view, and a security camera compilation of cars speeding by. I tagged my local representatives, and I asked for help.

While not all of them were receptive to my post, one particularly helpful representative connected me with my city’s City Engineer. The representative instructed me to send the City Engineer all of the evidence I had collected along with another copy of my Stop Sign Request Form.

The engineer was impressed with the code I wrote and the tracking system I’d put together, and she agreed to meet me at my house to do an inspection of the intersection. I accompanied her on the inspection so I could watch what she did. After working so hard to advocate for my community, it felt good to have my opinions heard.

In the end, I got my stop sign. Drivers still occasionally speed, but I was astounded by the outpouring of thanks I received after my neighborhood was alerted of the change. My foray into local government was an eventful but rewarding one. And even though I’ve secured my stop sign, I’ll still be doing stop sign research this summer—this time as an intern at the City Engineer’s office.
"""

In [40]:
#6: Fran’s Flower Farm, Grade: A

essay_A6 = """
Surrounded by carnations, dahlias, and marigolds, I laid down on the hard dirt, sweating from the midday sun. While my garden was a labor of love, it was still a labor. I’d spent months during the beginning of the pandemic researching how to set up beds correctly, choose seeds and fertilizers, and run a small business. A year later, this summer would be the second harvest of Fran’s Flower Farm.

As I prepared the yield for my small table at that week’s farmers market, I reflected on how far I had come. Prior to the pandemic, I had never even dug in the dirt. I didn’t know anything about seed germination or nitrogen levels. I had my own Instagram, but I had never had to market anything or think about overhead costs. I was a total and complete newb.

But my life, like everyone’s, changed in spring of 2020. Lockdown rendered me depressed and hopeless until one day when my mom ordered me a bouquet of flowers along with our grocery delivery. The bouquet was a simple grocery store arrangement of sunflowers. A few petals were wilting at the ends, and the stems were smashed from the flour that had been in the same plastic bag. But they were perfect. Such a small and thoughtful gesture, that bouquet inspired me to get to work.

Lucky enough to have space for flower beds, I mapped out four different six-foot beds in my backyard. Garden tools stolen from my mom and borrowed from socially-distanced neighbors in hand, I added compost, arranged my seeds, watered, and mulched. I laid protective plastic over my beds, tucking them in like a child, and wrapped the garden in decade-old chickenwire I found in our barn. My garden was imperfect–compost trailed between beds, my hose wrapped around my shovel in a heap on the ground, and the chickenwire was dented and rusty. But it was all mine, and it was alive.

As the pandemic waged on, I tended to my flowers. Each morning, I’d peek under the plastic to see how they had fared throughout the night. They gave me routine and purpose when the days seemed droning and neverending. The longer I kept them alive, the more their sprouts brought me life, too. In a world that seemed to come to a halt, my flowers showed me that growth wasn’t just possible–it was happening right in front of me.

The business side came soon after. Later that summer, once my first crop had bloomed, I set up a roadside stand outside of my house. At that point, I had to put my flower buckets across the driveway from my stand to keep everyone safe. But my flowers brightened the days of hundreds of passing motorists. With growing confidence, I secured a spot at the farmer’s market by July, my business boomed. Returning all profits to my garden, I’ve expanded my operations to include two more flower beds this year.

I’m proud of how far my gardening and business skills have come, but what has been most fulfilling about Fran’s Flower Farm have been the connections I’ve made. The pandemic was difficult for everyone, but it was especially difficult for healthcare workers. As the child of a healthcare worker myself, these challenges have been close to home. Knowing how greatly that bouquet of sunflowers affected me, I make sure to donate flowers to my local hospital in thanks every week.

Three years ago, I would never have guessed that I’d own my own flower farm. It’s brought me so many joys, challenges, and friends. I know I won’t be able to bring my flower farm with me to college. But the heart of the farm is more than the flowers. It’s about me learning and using my skills to help others. Wherever I’m planted, I know that I will bloom.
"""

In [41]:
#7: My Emotional Support Water Bottle, Grade: A-

essay_A7 = """
I had a stuffed animal named Elephant when I was a child. I’ve long since outgrown Elephant, but now I have a new object that I keep around for comfort: my emotional support water bottle. A gray thirty-two-ounce wide-mouth Hydroflask, my emotional support water bottle accompanies me everywhere.

The water bottle was a gift last Christmas after I begged my mom for one. The brand had become extremely popular at my school, and I wanted in on the trend. When I opened the package that Christmas morning, I was elated. I felt an immediate attachment, and I was proud that I could finally fit in with the other kids at my school.

I had always felt like an outsider. Other students seemed to fit together like puzzle pieces. But as much as I tried, I couldn’t find a picture that matched my piece. I envied the tight-knit friendships I saw among my peers.

As soon as I unwrapped my water bottle, I decided that I needed stickers to match. The kids at my school always had stickers on theirs. I found the perfect pack. It had animated depictions of every famous literary character imaginable. Jane Austen characters, Jay Gatsby, Sherlock Holmes, Guy Montag, Jane Eyre, and more. I couldn’t believe my luck.

No matter how disconnected I felt from my classmates, I could always find a community on my bookshelf. I sat in the courtroom with Atticus Finch, walked through the streets of Saint Petersburg with Raskolnikov, and watched the revolution unfold alongside Satrapi. My literary friends kept me optimistic through difficult times, and I was glad to see them every day on my beloved Hydroflask.

After winter break ended, I couldn’t wait to debut my new accessory. I placed it atop my desk in each class, angling my favorite stickers outward in hopes of connection. I was profoundly comforted by its presence—I could always take a sip of water when I felt thirsty or uncomfortable, and its stickers promised to draw people in.

To my dismay, weeks went by, and no one noticed my Hydroflask or stickers. The school was filled with dozens more Hydroflasks after the holidays, so mine didn’t seem so special. What had once filled me with so much hope and support transformed into a reminder of an unfulfilled promise of friendship.

I coped with the disappointment by re-reading one of my childhood favorites, Le Petit Prince. Near the end, when the little prince returns to water his flower, I had a realization. I couldn’t wait around for people to come to me. I had to bring the water to them.

The next day at school, I held my Hydroflask close and gathered all my courage. I headed into the lunch room and spotted Jordan, one of the people I’d chatted with in class. She was sitting alone at a table, reading a book I couldn’t identify. I asked if I could join her. Nodding, she told me about her book, White Teeth. When I placed my Hydroflask on the lunch table, she noticed my stickers. Together, we went through every sticker and talked about the character’s book.

Jordan and I spent the next day’s lunch exchanging laughter and book recommendations. She had a water bottle of her own, too. It was a classic Nalgene without a single sticker. As our friendship grew stronger, I brought Jordan the last sticker from my collection, a rainbow bookmark that read, “BOOKWORM.”

I’ve always looked to the world around me for comfort instead of finding courage within myself. Elephant still sits on my shelf, I continue to be an avid reader, and I always carry my Hydroflask around for hydration. But this learning process has taught me the importance of having confidence and finding the ability to reach out to others. I can’t wait to carry this skill with me to college—after I get some more stickers.
"""

In [42]:
#8: Party of One , Grade: A-

essay_A8 = """
The sun shone through my airplane window, hitting the tray table exactly right to reveal the greasy handprint of a child. Beside me, a woman cleared her throat as she rifled through her purse, and the tween next to her tapped away on an iPad. The knees of the tall man behind me pushed against the back of my chair. Together, we headed to Pennsylvania.

This wasn’t my first trip to Pennsylvania, and it wouldn’t be my last. But it was my first trip traveling as a party of one. Barely past the unaccompanied minor cutoff, I departed for a month-long and court-ordered trip to my dad’s house. I wasn’t eager to travel alone. I felt afraid, too young to do this by myself. I wanted to go back home. But I decided to embrace the journey as an adventure.

With the growing whirr of the engines, the plane ascended. All around me, my neighbors breathed sighs of relief when we reached cruising altitude. I tightened my seatbelt across my lap, steadying myself for the five-hour trip, and took in the scene. Always the quiet and careful observer, a full flight was my Sistine Chapel.

The woman to my right was wearing all black. She extracted her laptop from her bag the moment the flight attendants permitted, and she created a PowerPoint presentation from scratch before the drinks cart had even started down the aisle. She was all business. I imagined that she signed her emails with nothing but her name, that she read Keynes in her free time, and that people listened when she spoke. She was everything I longed to be.

Next was the tween, only a few years younger than I was. Clearly afraid of flying, the tween reached across the aisle to a man who was presumably her father. I found it endearing that she reached out in fear. The dad’s reassurance didn’t just comfort the tween. It comforted me. So far from home, his quiet calm reminded me of the parent waiting to pick me up at the other end of this journey. I remembered reaching out for my own father’s hand when we flew to Pennsylvania for the first time. Now, I watched the dad squeeze the tween’s hand. I felt guilty for the frustration I felt about the trip. I was excited to see my dad.

And finally, there was the man behind me. Aside from the brief glimpse I got during boarding, I didn’t know what he looked like. But there were two things I knew to be true. First, he was tall. The longer the flight went on, the more apologetically his knees bumped against my seat. Second, I felt emboldened by his ability to take up space. With each nudge forward, I spread myself a little bigger, daring to exist in a world I normally wanted to hide from.

Four hours into the flight, turbulence hit. The long-legged man yelped as his knee hit the metal of the seat. Bigger now, I was able to brace myself against the impact. I looked to the tween, who I expected to be a wreck. Instead, I saw a calm girl handing napkins to her dad, whose drink had spilled in the commotion. Her care for him mirrored the care he had shown for her. The woman next to me, who had seemed so steadfast, gasped when the plane shot downward. Her hand reached for her chest as she caught herself, surprised. I moved my arm from our shared armrest, giving her space. She smiled in appreciation.

After the turbulence had ended, I looked at myself. My hands were folded neatly in my lap. I realized that although I was flying solo, I was surrounded by strangers whose stories intersected with my own. When we landed, I ran into my dad’s arms. “You’ve grown,” he smiled
"""

In [43]:
#9: My Greatest Talent, Grade: B+

essay_B9 = """
I’m a klutz—that’s it, that’s my greatest talent. I’ve honed my clumsiness to perfection, putting in more than my 10,000 hours over the last… 17 years of my life.

When I was six or seven, I was always the one tripping over my own feet, knocking things over. (“This is why we can’t have nice things!” my mom used to scream, half in jest and half in exasperation.) My parents used to joke that I was the only person who could trip on a flat surface. But unfortunately for me, despite doing my due diligence into flat-earth theory, I found that there was a prevailingly devilish curve to everything around me. If it had a lip, an edge, or a slick spot, I found it.

As I got older, my talent for being a klutz grew. I managed to trip over my own backpack on a daily basis, and I once fell down a flight of stairs while holding a tray of cookies (I was trying to be a good hostess, but it didn't end well). My friends and family came to expect it, and after those first few years of irritated glances, they began to meet my clumsiness with a laugh and an extended hand.

Being a klutz isn't all bad. In fact, it has some pretty decent perks. For one thing, it’s helped me become more empathetic. I know what it feels like to stumble and fall (and stumble and fall, and stumble and fall, and…), and I’m always ready to offer a kind word and a hug to someone who’s having a tough time. I also have a great sense of humor—a defense mechanism thanks to all of the embarrassing moments that I’ve created for myself. And let's not forget the fact that I am never bored. There is always something to trip over or knock over. Neither I nor anyone around me ever lacks for entertainment.

One of the biggest benefits of being a klutz is the unexpected friendships it has given me. For example, I once tripped and fell into a ditch while hiking with a group of near-strangers I had met at a trailhead. Surrounded by brambles and thorns, three of them jumped right down with me to hoist me out. My graceless tumble became an inside joke of the trip and we all ended up becoming good friends. I was still embarrassed, of course, but I’m grateful that my clumsiness opened up a new door for friendship that day.

Being a klutz has also taught me to be patient with myself, and to not take myself too seriously. It has taught me to always be prepared for the unexpected, and to always have a good sense of humor. And most importantly, it has taught me to be kind to others, especially when they are having a tough time.

So, if you are looking for someone who’s a little bit quirky and a lot of fun, I’m your girl. I may not be the most graceful person on the planet, or on your campus, but I am confident, kind, and always up for a good laugh. Anyway, where's the fun in being graceful? Just, please, if you do accept me—I’d really appreciate some foam bumpers on the sharp surfaces in my dorm.
"""

In [44]:
#10: Counting Cards, Grade B

essay_B10 = """
I am a psychic who thinks in terms of fours and threes. Deal me any hand of Gin, and I can guarantee I’ll have you beat. I stare at the cards in my hand and see numbers moving in my mind. Like a mathemetician at a chalkboard, I plan out my next move. I use logic, memory, and a little bit of luck to guess exactly what your hand looks like. The possible combinations seem endless—four Kings and a run of three, three nines and four Queens, a run of four and three sevens, and many, many more. What I love most about playing Gin is the predictability. While I may not know what’s coming, I can use what I already know to strategize, adapt, and have fun along the way.

My Gin career began as a small child. My aunt taught me how to play the game while we were camping. My hands were so small that we had to use a chip clip to keep the cards in place. I was at first intimated by the “big kid game,” as I called it then, but soon I couldn’t get enough. I forced my entire family to play, and I even roped in the kids at the campsite next to us. My aunt, a mathematician, is a skilled Gin player. She passed her tips and tricks along to me. After a few years of playing, she was the only opponent I couldn’t beat.

Last summer was the first time it finally happened. I bested her. I had a hand with three Aces and a run of Spades. I needed another Ace or a three or seven of Spades. When I drew that final Ace from the deck, I could hardly believe it. I paused to count my cards again. I drew my hands to my chest, looked up at my aunt slowly and triumphantly, and calmly declared, “Gin.” My aunt squealed and embraced me, proud of all the progress her protegee had made.

This win came from a year of hard work. I read every book on Gin I could find at the library, watched countless YouTube videos, and became an expert on Gin’s more lively counterpart, Gin Rummy. Learning and practicing drew me into a huge online community of Gin enthusiasts. I never thought that I’d meet some of my best friends through a card game, but I did. Every night, we’d compete against each other. And with each match, my skills would sharpen like a knife on a honing steel. When I finally beat my aunt, I hadn’t just won the game. I’d won lifelong friends and greater reasoning skills.

Gin players aren’t internationally recognized for their intellectual prowess like chess or Scrabble. I’ve learned other games and played them successfully, but nothing has come close to the joy and challenge I feel while playing Gin. I love predicting what your opponent holds and what you’ll draw next, betting on your perfect card being in the draw deck, chatting with your opponent as you deal the next round, and earning bragging rights after winning a match—all of it is the perfect mix of strategy and community. When I head off to college in the fall, the first thing I’ll pack will be a deck of cards.
"""

In [45]:
#11: Golden Hills Animal Clinic, Grade: B+

essay_B11 = """
On my best days at work, I’m surrounded by puppies, kittens, and rainbows. On my worst, I watch people say tearful goodbyes to their best friends. Working at the front desk of Golden Hills Animal Clinic, I’ve seen it all. I’ve learned a lot about people through their pets. I’ve also learned a lot about myself.

I began working in the clinic two summers ago. I’m known in my family as the “Snow White” because I’ve always had a special connection with animals. I had nearly started a new colony of stray cats in my backyard by the time I was nine. I’ve nursed more sick and injured birds than I can count. I’ve discovered all kinds of insects, snakes, and lizards in my neighborhood. Now, at the front desk, I get to welcome the animals and their humans. I share in their joys and console them at their lows.

After watching thousands of animals struggle, you think you’d get used to the pain and suffering. But each hurt, injured, or elderly animal I check in stings just the same. When I’m in the back room helping prepare the animals for surgeries or procedures, I look into their eyes and desperately try to communicate that everything will be okay. The worst part is knowing that the animals can tell something is wrong but don’t understand what is happening. And when their owners walk past my front desk, I reassure them that we’re treating their pets as our own.

But with life’s hard moments also come the happiest ones. It’s easy to become dejected by the sad times, but working at the clinic has actually given me more hope. There’s nothing like seeing small puppies, feet too big for their bodies, prance through the waiting room. I’ve witnessed children comfort cats through holes in carriers, and I’ve become inspired by the assertiveness with which our veterinarians make critical decisions to help animals. Through all this, I’ve learned that those little pockets of happiness, care, and determination are what make life worth living.

I’ve also learned that veterinary medicine is as much about the people as it is the pets. Sometimes owners have to be convinced about the best care plan for their pets. Sometimes others aren’t able to afford the care they desperately want to get. People come in worried about nothing or not worried enough. Part of managing the front desk is having the ability to read where a person is coming from the moment they start speaking. Seeing things from customers’ perspectives helps me provide better customer service to the people and the pets. If I sense that a customer is worried about cost, I can talk to them about payment plans. If someone seems overwhelmed by the options, I ask if they’d like to speak with the vet again. In all these cases, I feel proud to provide as much help as I can. Doing so makes sure that our animals receive the best care possible.

Now, as an aspiring veterinarian myself, I know that the rest of my career will be filled with the happiest and saddest moments of people’s lives. My care for animals will turn tragedies into miracles. I’ll console owners of sick pets, and I’ll help bring new life into the world. Veterinary medicine is a lot like life in general. You can’t have the good without the bad. But I’ve never met a pet owner who wouldn’t trade the pain of animal loss for even one fleeting, happy moment with their furry friend. Animals make the world a better place. Like Snow White, I’ll continue listening to animals so I can make their world a little better too.
"""


In [46]:
#12: The Filmmaker, Grade: B

essay_B12 = """
Eye to the lens, I feel in complete control. The old camera weighs heavy in my hands as I quietly point my leading actor to the other side of the frame. Taking a moment to look at the world through my own eyes rather than a lens, I make a decision. I back up, careful not to trip, and capture the wide, panning shot I had envisioned. Filmmaking allows me to show others exactly how I see the world. With an odd angle or lingering aside, I can take my audience on a journey through my eyes.

What’s beautiful about filmmaking is that there are several art forms occurring simultaneously. At the foundation of a scene is the script. Words that draw a viewer in and keep them there, the script is an essential act of creative writing. Next there’s the acting. An art of performance, acting brings the script to life. A good actor will make an audience feel as if they are with the characters, feeling what they feel and doing what they do. Then there’s the direction and filmmaking. Choices about how to translate a three-dimensional world to pixels on a screen drastically affect the audience’s experience. And, finally, there’s the editing. Editing is where all of the other art forms converge, selected and chopped up and stitched back together to create something even better than the original.

I’ve never been one for writing or acting. But the latter two, filmmaking and editing, are where my passions lie. Inspired by my favorite movie, ET, I began filmmaking in elementary school. Borrowing my mom’s Flip UltraHD camera, I’d run around my home, filming everything in sight. Soon after, I started gathering my neighborhood friends in my backyard and directing them in made-up film productions. Our films took us on journeys around the world. We were pirates in the Atlantic, merchants in Paris, and kangaroos in Australia. We learned how to tell stories and create and resolve conflicts. In the process, we learned about ourselves, each other, and the world around us.

My love for editing didn’t come until later. When my family upgraded our ancient Gateway 2000 to a sleek iMac, I became an iMovie aficionado. I learned how to use all the features and enter in keyboard shortcuts. I became a sculptor. Instead of clay, my material was digital. I’d split clips in half, manually zoom in to my subject, and add filters that changed the whole tone of a shot. Shift + Command + F, and I’d play my clips in full screen, evaluating them with the eye of a film critic. Was my shot effective? Are the actors convincing? Is there anything odd in the background? If I had never seen this, what would I think and feel? Then I’d repeat the process, over and over again.

Some people might say that dedicating myself to filmmaking is frivolous in a world with more pressing problems. But filmmaking is a way to spread messages and give people hope. From the change wrought by An Inconvenient Truth to the laughter Mr. Bean has incited in millions, filmmaking is a way to bring art, truth, and laughter to everyone. More accessible than books or newspapers, film and TV couldn’t be more essential media to confront the problems of today. With the passion of my ten-year-old self, the films I’ll continue to make will have an impact.
"""

In [47]:
#13: Lost in the Forest, Grade: D

essay_D13 = """
I look into the forest, moss wet on my feet. There’s fog everywhere—I can barely see the glasses that sit on my nose. I feel a cool breeze rustle against my coat. I am cold and warm all at once. The sun shines through the fog, casting the shadow of a tree whose roots know no end. At the entrance to the forest, I stand frozen in time and space. I can’t see what’s ahead of me or behind me, only what is. And what is suddenly transforms into what could be. I see a fork in the pathway in front of me. The noise—the noise is so loud. Crickets and owls and tigers, oh my. My thoughts scream even louder. I can’t hear myself think through the sounds of the forest of my mind. Off in the distance, I see a figure. It’s a shadow figure. It’s my mother. She’s walking towards me. I take a step into the forest, fearlessly ready to confront any overwhelming obstacle that comes my way.

When I was a child, I used to play in the forest behind my house. Until one day when I caught my mom sneaking a cigarette outside. She tried to hide it behind her back, but I could see the smoke trailing over her head like a snail. I didn’t know what to do, so I ran farther into the forest. I am used to being disappointed by her. I ran and ran and ran until I tripped over a tree branch that fell in the storm the week before. I laid on the cold, hard ground. The back of me was soaked. Would I turn into my mom? After that, I decided to turn back. The cold was encroaching. I got home and saw my mom in the kitchen. We agreed not to speak of what I saw.

While taking a history test, I looked around at my classmates. The gray desk was cold against my skin. I started counting the people around me, noting those who I knew well and those I had never really talked to. I looked at all the expensive backpacks and shoes. After our test, I asked the person next to me how she thought she did. She said it was a difficult test, and I agreed. Every class period, we’d talk more and more. We became friends. We started hanging out with another friend from biology class. We were inseparable, like three peas in a pod. We’d study together and hang out together and dance. They were the best friends I ever had. We liked to play soccer after school and sing loudly to music in my room. But one day it all stopped. They both stopped talking to me. It was like I had been yanked out of the forest and thrown on to the forest floor. I became moss, the owls pecking at my spikey green tendrils. They found two other friends, and I sat alone at my desk in history again. It was like another test, but this time a history of my own.

Things went on like this for years. Over and over again I got put back into the forest. My friends who I thought were my friends actually were just drama machines. Life is foggy when you don’t know what’s going on. And I live in a forest that’s always foggy. Try as I might to find myself, it’s easy to get lost in all the trails and hills. I’m climbing a mountain each and every day. But I keep going back into the forest, looking for answers.
"""

In [48]:
#14: The Chemist, Grade: F

essay_F14 = """
You. may be wondering why I’ve taken so many chemistry classes. 
Well, that’s because I love chemistry. 
I used to hate chemistry with a fiery passion but now I love it more than anything. 
I remember that I used to struggle through every single chemistry assignment I ever got. 
My sister would try to help me but I’d just get upset, like I really just didn’t understand it 
and that was so frustrating so I just kept not wanting to do more but eventually 
I started to think “oh chemistry is at the foundation of everything that makes up our universe,” 
and isn’t that just fascinating? So then I decided to make a change and actually try to learn chemistry. 
I started paying attention in class and asking my teacher for help after class and finally one day my sister said, 
“Wow, you’re really improving.” And that meant so much to me. 
When my great-grandparents immigrated to the United States, they had no idea what would be in store 
for their great-grandkids. We really don’t learn chemistry in school until high school, 
so it’s no wonder I didn’t understand it in high school when I started taking it. 
Electrons and atoms and acids and alcohols. There’s so much to learn. I really have never been good at math 
so I’d say that’s one of my biggest challenges in chemistry now is learning how to do the equations 
and figuring out how the math works. In fifth grade I used to be in advanced math but then 
it just got worse from there until I learned about tutoring. I started doing tutoring through the high school when 
I was in ninth grade and it helped a lot because I just needed a little more help 
for each lesson to really understand it. But even with that the math part of chemistry is still hard for me. 
But I always keep trying! That’s the most important thing to me I think is to keep trying. 
Even when problems are hard and I can’t solve them I try to have a good attitude because even if 
I can’t get it right, doing chemistry is about unlocking the secrets of the universe 
and that really is interesting even if you can’t completely understand them. 
When I started taking chemistry in my sophomore year I almost gave up but 
I was also really inspired by my teacher who guided me through everything. 
She gave me extra time to do my lab work and was even my lab partner a couple times because our 
class has an uneven number of students. My favorite part of chemistry lab is mixing solutions and testing them. 
I don’t like the lab report writing so much but I know it’s an important part. 
So I try to just get through that so I can get back to doing experiments and such. 
My favorite experiments was about building a calormieter to measure how many calories is in our food. 
Calories are energy so you burn your food to measure how much energy they have. 
Then you write up a report about how many calories each food item like bananas, bread, a cookie, had. 
The best part of doing labs is having your lab partner there with you. You’re both wearing goggles 
and lab coats and gloves and you feel really like a professional chemist and it’s nice that you’re not doing 
it alone. You just read the lab instructions and do each of the steps in order. It’s like baking a cake! 
You just follow the recipe. But you don’t eat the results! You might use beakers or bunsen burners 
to hold liquid or burn or heat up whatever it is you’re experimenting on. 
And when I say “find the meaning of the universe” I really mean it. 
It’s amazing how much chemistry is in everything. Cooking is doing chemistry because you’re changing up 
the properties of the food. The air we breathe, the way plants get energy, 
the medicines we take, we understand it all because of chemistry. I know that becoming a chemist is hard work 
and isn’t easy. But I know that it’s rewarding and that’s why I want to do it. Helping people is so important 
to me and I think that chemistry can help me get there. I also like the health and beauty industry and 
I think it would be fun to get to develop new products or perfumes or medicines.
"""

## Data preparation

In [1]:
system_message = f"""You are a helpful teacher assistant that grades a students essay,
                the grading has the list of rubrics: {rubrics} 
                and criteries of grading in each rubrics : {json.dumps(criteries, indent=4)}
                """


NameError: name 'rubrics' is not defined

In [161]:

def prepare_example_conversation(essay, score, system_message=system_message):
    """
    Create the data of QA the model.
    """
    messages = []
    messages.append({"role": "system", "content": system_message})

    messages.append({"role": "user", "content": essay})

    messages.append({"role": "assistant", "content": get_feedback(score)})

    return {"messages": messages}

print(prepare_example_conversation(essay_A1, 6))

{'messages': [{'role': 'system', 'content': 'You are a helpful teacher assistant that grades a students essay,\n                the grading has the list of rubrics:  Organization, Content, Grammar, Vocabulary, Overall impression  \n                and criteries of grading in each rubrics : {\n    "Organization": {\n        "1": "Essay lacks clear structure and coherence. Ideas are disorganized and difficult to follow.",\n        "2": "Essay has some organization, but transitions between ideas are weak. Some parts may be unclear or confusing.",\n        "3": "Essay has a logical structure with clear transitions between ideas. Most parts are clear and easy to follow.",\n        "4": "Essay is well-organized with strong transitions between ideas. All parts are clear and coherent.",\n        "5": "Essay has a highly effective organization with seamless transitions between ideas. All parts are clear, coherent, and engaging.",\n        "6": "Essay demonstrates exceptional organization. Ideas

### Make the train data.

In [162]:
## Scores of the essays on the web site

# 1: The Gospel of Steve,  Grade:A+
#2: The Embroidery Scientist, Grade:A
#4: Poetry Slam, Grade:A
#5: The Stop Sign, Grade: A+
#6: Fran’s Flower Farm, Grade: A
#7: My Emotional Support Water Bottle, Grade: A-
#8: Party of One , Grade: A-
#9: My Greatest Talent, Grade: B+
#10: Counting Cards, Grade B
#11: Golden Hills Animal Clinic, Grade: B+
#12: The Filmmaker, Grade: B
#13: Lost in the Forest, Grade: D
#14: The Chemist, Grade: F

# Write the scores in numbers
essays_names = {"essay_A1" : (essay_A1, 6.0),
                "essay_A2" : (essay_A2, 5.8), 
                "essay_A4" : (essay_A4, 5.8), 
             #   "essay_A5" : (essay_A5, 6.0),
                "essay_A6" : (essay_A6, 5.8),
                "essay_A7" : (essay_A7, 5.5), 
                "essay_A8" : (essay_A8, 5.6), 
                "essay_B9" : (essay_B9, 5.3),
             #   "essay_B10": (essay_B10, 5.0),
                "essay_B11": (essay_B11, 5.2),
                "essay_B12": (essay_B12, 5.0),
            #    "essay_D13": (essay_D13, 3.0),
                "essay_F14": (essay_F14, 1.0)
               }
v_essays_names = {"essay_A5" : (essay_A5, 6.0),
                 "essay_B10": (essay_B10, 5.0),
                 "essay_D13": (essay_D13, 3.0),
                 }

training_data = []
                
for k,v  in essays_names.items():
    
    print(k)
    training_data.append(prepare_example_conversation(v[0], v[1]))
    
validation_data = []
                
for k,v  in v_essays_names.items():
    print(k)
    validation_data.append(prepare_example_conversation(v[0], v[1]))
    

essay_A1
essay_A2
essay_A4
essay_A6
essay_A7
essay_A8
essay_B9
essay_B11
essay_B12
essay_F14
essay_A5
essay_B10
essay_D13


In [163]:

def write_jsonl(data_list: list, filename: str) -> None:
    """
    Write a training file
    """
    with open(filename, "w") as out:
        for ddict in data_list:
            jout = json.dumps(ddict) + "\n"
            out.write(jout)

In [164]:
training_file_name = "tmp_essay_grading_training.jsonl"
write_jsonl(training_data, training_file_name)

validation_file_name = "tmp_essay_grading__validation.jsonl"
write_jsonl(validation_data, validation_file_name)

### Upload files

In [165]:

client = openai.OpenAI(api_key=OPENAI_API_KEY )

training_response = client.files.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
training_file_id = training_response.id


validation_response = client.files.create(
    file=open(validation_file_name, "rb"), purpose="fine-tune"
)
validation_file_id = validation_response.id

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-2jcyfHERImLihwAk4efbMNoX
Validation file ID: file-QlwlkSSvl97UHrEx7j1C9CCW


In [64]:
#!openai migrate

Your working tree currently has untracked changes and Grit will rewrite files in place. Do you want to proceed? [y/N] 

## Fine tuning

In [168]:
response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model="gpt-3.5-turbo-0613",  # gpt-3.5-turbo-1106
    suffix="essay-grading",
    hyperparameters = {"batch_size": 1,
                       "learning_rate_multiplier": 1e-4,
                       "n_epochs": 1}
)

job_id = response.id

print("Job ID:", response.id)
print("Status:", response.status)

Job ID: ftjob-7W2BXDYedY8d8zaYyotK4IW7
Status: validating_files


In [173]:
response = client.fine_tuning.jobs.retrieve(job_id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-7W2BXDYedY8d8zaYyotK4IW7
Status: running
Trained Tokens: None


In [182]:
response = client.fine_tuning.jobs.list_events(fine_tuning_job_id=job_id, limit=50)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Created fine-tuning job: ftjob-7W2BXDYedY8d8zaYyotK4IW7
Validating training file: file-2jcyfHERImLihwAk4efbMNoX and validation file: file-QlwlkSSvl97UHrEx7j1C9CCW
Files validated, moving job to queued state
Fine-tuning job started
Step 1/100: training loss=0.18, validation loss=0.21
Step 11/100: training loss=0.02, validation loss=0.02
Step 21/100: training loss=0.02, validation loss=0.03
Step 31/100: training loss=0.01, validation loss=0.01
Step 41/100: training loss=0.02, validation loss=0.01
Step 51/100: training loss=0.01, validation loss=0.07
Step 61/100: training loss=0.00, validation loss=0.01
Step 71/100: training loss=0.01, validation loss=0.01
Step 81/100: training loss=0.01, validation loss=0.10
Step 91/100: training loss=0.00, validation loss=0.00
New fine-tuned model created: ft:gpt-3.5-turbo-0613:constructor-tech:essay-grading:8LdcdGZ7
The job has successfully completed


In [183]:
response = client.fine_tuning.jobs.retrieve(job_id)
fine_tuned_model_id = response.fine_tuned_model

if fine_tuned_model_id is None: 
    raise RuntimeError("Fine-tuned model ID not found. Your job has likely not been completed yet.")

print("Fine-tuned model ID:", fine_tuned_model_id)

Fine-tuned model ID: ft:gpt-3.5-turbo-0613:constructor-tech:essay-grading:8LdcdGZ7


In [225]:
fine_tuned_model_id = "ft:gpt-3.5-turbo-0613:constructor-tech:essay-grading:8IyoJ5r6"
fine_tuned_model_id = "ft:gpt-3.5-turbo-0613:constructor-tech:essay-grading:8IzElULZ"

fine_tuned_model_id = "ft:gpt-3.5-turbo-1106:constructor-tech:essay-grading:8LbnXV8h"

### Inferaance

In [184]:
d_test = {"essay_A5" : (essay_A5, 6.0),
                 "essay_B10": (essay_B10, 5.0),
                 "essay_D13": (essay_D13, 3.0),}

In [185]:
test_messages = []
test_messages.append({"role": "system", "content": system_message})

test_messages.append({"role": "user", "content": d_test['essay_A5'][0]})

In [186]:
def get_completion(messages, model=llm_model):
    
    try:
        from openai import ChatCompletion
        
        response = ChatCompletion.create(
                   model=model,
                   messages=messages,
                   temperature=0, 
                    )
        
        return response#.choices[0].message["content"]
    
    except:
        from openai import OpenAI
        
        client = OpenAI(api_key=OPENAI_API_KEY )
        response = client.chat.completions.create(
                          model=model,
                   messages=messages,
                   temperature=0 
                   )
        return response.choices[0].message.content  
    


In [187]:
response = get_completion(test_messages, model="ft:gpt-3.5-turbo-1106:constructor-tech:essay-grading:8LbnXV8h")

print(d_test['essay_A5'][1])
print(response)

6.0
{
    "Organization": {
        "6": "Essay demonstrates exceptional organization. Ideas flow logically and coherently. The structure enhances the overall impact of the essay."
    },
    "Content": {
        "6": "Essay showcases exceptional and profound content. Ideas are exceptionally developed, insightful, and demonstrate exceptional originality."
    },
    "Grammar": {
        "6": "Essay is virtually flawless in terms of grammar. Sentence structure and word choice are impeccable."
    },
    "Vocabulary": {
        "6": "Essay showcases an exceptional range of vocabulary with precision and sophistication. Word choice is impeccable."
    },
    "Overall impression": {
        "5": "Essay leaves a highly positive overall impression. It is exceptionally well-organized, coherent, and developed."
    }
}


In [188]:
test_messages = []
test_messages.append({"role": "system", "content": system_message})

test_messages.append({"role": "user", "content": d_test['essay_B10'][0]})

In [189]:

response = get_completion(test_messages,  model=fine_tuned_model_id)

print(d_test['essay_B10'][1])
print(response)

5.0
{
    "Organization": {
        "5": "Essay has a highly effective organization with seamless transitions between ideas. All parts are clear, coherent, and engaging."
    },
    "Content": {
        "6": "Essay showcases exceptional and profound content. Ideas are exceptionally developed, insightful, and demonstrate exceptional originality."
    },
    "Grammar": {
        "5": "Essay has virtually no grammatical errors. Sentence structure and word choice are consistently correct."
    },
    "Vocabulary": {
        "5": "Essay exhibits a wide range of vocabulary with precision and sophistication. Word choice is consistently correct and clear."
    },
    "Overall impression": {
        "5": "Essay leaves a highly positive overall impression. It is exceptionally well-organized, coherent, and developed."
    }
}


In [190]:
test_messages = []
test_messages.append({"role": "system", "content": system_message})

test_messages.append({"role": "user", "content": d_test['essay_D13'][0]})

In [191]:

response = get_completion(test_messages, model=fine_tuned_model_id)

print(d_test['essay_D13'][1])
print(response)

3.0
{
    "Organization": {
        "1": "Essay lacks clear structure and coherence. Ideas are disorganized and difficult to follow."
    },
    "Content": {
        "1": "Essay lacks relevant and meaningful content. Ideas are superficial and lack depth."
    },
    "Grammar": {
        "1": "Essay has numerous grammatical errors that impede understanding. Sentence structure and word choice are consistently incorrect."
    },
    "Vocabulary": {
        "1": "Essay has limited and inappropriate vocabulary. Word choice is often incorrect or unclear."
    },
    "Overall impression": {
        "1": "Essay leaves a negative overall impression. It lacks coherence, depth, and clarity."
    }
}


## Fine tuning second step

In [192]:
## Scores of the essays on the web site

# 1: The Gospel of Steve,  Grade:A+
#2: The Embroidery Scientist, Grade:A
#4: Poetry Slam, Grade:A
#5: The Stop Sign, Grade: A+
#6: Fran’s Flower Farm, Grade: A
#7: My Emotional Support Water Bottle, Grade: A-
#8: Party of One , Grade: A-
#9: My Greatest Talent, Grade: B+
#10: Counting Cards, Grade B
#11: Golden Hills Animal Clinic, Grade: B+
#12: The Filmmaker, Grade: B
#13: Lost in the Forest, Grade: D
#14: The Chemist, Grade: F

# Write the scores in numbers
essays_names = {"essay_A1" : (essay_A1, 6.0),
                "essay_A2" : (essay_A2, 5.8), 
                "essay_A4" : (essay_A4, 5.8), 
             #   "essay_A5" : (essay_A5, 6.0),
                "essay_A6" : (essay_A6, 5.8),
                "essay_A7" : (essay_A7, 5.5), 
                "essay_A8" : (essay_A8, 5.6), 
                "essay_B9" : (essay_B9, 5.3),
             #   "essay_B10": (essay_B10, 5.0),
                "essay_B11": (essay_B11, 5.2),
                "essay_B12": (essay_B12, 5.0),
                "essay_D13": (essay_D13, 3.0),
                "essay_F14": (essay_F14, 1.0)
               }
v_essays_names = {"essay_A5" : (essay_A5, 6.0),
                 "essay_B10": (essay_B10, 5.0),
             #    "essay_D13": (essay_D13, 3.0),
                 }

training_data = []
                
for k,v  in essays_names.items():
    
    print(k)
    training_data.append(prepare_example_conversation(v[0], v[1]))
    
validation_data = []
                
for k,v  in v_essays_names.items():
    print(k)
    validation_data.append(prepare_example_conversation(v[0], v[1]))
    

essay_A1
essay_A2
essay_A4
essay_A6
essay_A7
essay_A8
essay_B9
essay_B11
essay_B12
essay_D13
essay_F14
essay_A5
essay_B10


In [193]:

def write_jsonl(data_list: list, filename: str) -> None:
    """
    Write a training file
    """
    with open(filename, "w") as out:
        for ddict in data_list:
            jout = json.dumps(ddict) + "\n"
            out.write(jout)

In [194]:
training_file_name = "tmp_essay_grading_training.jsonl"
write_jsonl(training_data, training_file_name)

validation_file_name = "tmp_essay_grading__validation.jsonl"
write_jsonl(validation_data, validation_file_name)

In [195]:
response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model=fine_tuned_model_id,
    suffix="essay-grading",
)

job_id = response.id

print("Job ID:", response.id)
print("Status:", response.status)

Job ID: ftjob-PCSWWcGsNXKfZ92XxaAVlGWg
Status: validating_files


In [196]:
response = client.fine_tuning.jobs.retrieve(job_id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-PCSWWcGsNXKfZ92XxaAVlGWg
Status: validating_files
Trained Tokens: None


In [199]:
response = client.fine_tuning.jobs.list_events(fine_tuning_job_id=job_id, limit=50)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Created fine-tuning job: ftjob-PCSWWcGsNXKfZ92XxaAVlGWg
Validating training file: file-2jcyfHERImLihwAk4efbMNoX and validation file: file-QlwlkSSvl97UHrEx7j1C9CCW
Files validated, moving job to queued state
Fine-tuning job started
Step 1/100: training loss=0.01, validation loss=0.00
Step 11/100: training loss=0.01, validation loss=0.01
Step 21/100: training loss=0.00, validation loss=0.16
Step 31/100: training loss=0.00, validation loss=0.00
Step 41/100: training loss=0.02, validation loss=0.00
Step 51/100: training loss=0.00, validation loss=0.31
Step 61/100: training loss=0.00, validation loss=0.00
Step 71/100: training loss=0.00, validation loss=0.05
Step 81/100: training loss=0.00, validation loss=0.35
Step 91/100: training loss=0.00, validation loss=0.00
New fine-tuned model created: ft:gpt-3.5-turbo-0613:constructor-tech:essay-grading:8Ldp8RDs
The job has successfully completed


In [200]:
response = client.fine_tuning.jobs.retrieve(job_id)
fine_tuned_model_id = response.fine_tuned_model

if fine_tuned_model_id is None: 
    raise RuntimeError("Fine-tuned model ID not found. Your job has likely not been completed yet.")

print("Fine-tuned model ID:", fine_tuned_model_id)

Fine-tuned model ID: ft:gpt-3.5-turbo-0613:constructor-tech:essay-grading:8Ldp8RDs


### Inferaance

In [201]:
d_test = {"essay_A5" : (essay_A5, 6.0),
                 "essay_B10": (essay_B10, 5.0),
                 "essay_D13": (essay_D13, 3.0),
                 "essay_F14": (essay_F14, 1.0),
         }

In [202]:
test_messages = []
test_messages.append({"role": "system", "content": system_message})

test_messages.append({"role": "user", "content": d_test['essay_A5'][0]})

In [203]:
response = get_completion(test_messages, model=fine_tuned_model_id)

print(d_test['essay_A5'][1])
print(response)

6.0
{
    "Organization": {
        "6": "Essay demonstrates exceptional organization. Ideas flow logically and coherently. The structure enhances the overall impact of the essay."
    },
    "Content": {
        "6": "Essay showcases exceptional and profound content. Ideas are exceptionally developed, insightful, and demonstrate exceptional originality."
    },
    "Grammar": {
        "6": "Essay is virtually flawless in terms of grammar. Sentence structure and word choice are impeccable."
    },
    "Vocabulary": {
        "6": "Essay showcases an exceptional range of vocabulary with precision and sophistication. Word choice is impeccable."
    },
    "Overall impression": {
        "6": "Essay leaves an outstanding overall impression. It is exceptionally well-organized, coherent, developed, and engaging."
    }
}


In [204]:
test_messages = []
test_messages.append({"role": "system", "content": system_message})

test_messages.append({"role": "user", "content": d_test['essay_B10'][0]})

In [205]:
response = get_completion(test_messages, model=fine_tuned_model_id)

print(d_test['essay_B10'][1])
print(response)

5.0
{
    "Organization": {
        "5": "Essay has a highly effective organization with seamless transitions between ideas. All parts are clear, coherent, and engaging."
    },
    "Content": {
        "6": "Essay showcases exceptional and profound content. Ideas are exceptionally developed, insightful, and demonstrate exceptional originality."
    },
    "Grammar": {
        "5": "Essay has virtually no grammatical errors. Sentence structure and word choice are consistently correct."
    },
    "Vocabulary": {
        "5": "Essay exhibits a wide range of vocabulary with precision and sophistication. Word choice is consistently correct and clear."
    },
    "Overall impression": {
        "5": "Essay leaves a highly positive overall impression. It is exceptionally well-organized, coherent, and developed."
    }
}


In [206]:
test_messages = []
test_messages.append({"role": "system", "content": system_message})

test_messages.append({"role": "user", "content": d_test['essay_D13'][0]})

In [207]:
response = get_completion(test_messages, model=fine_tuned_model_id)

print(d_test['essay_D13'][1])
print(response)

3.0
{
    "Organization": {
        "5": "Essay has a highly effective organization with seamless transitions between ideas. All parts are clear, coherent, and engaging."
    },
    "Content": {
        "6": "Essay showcases exceptional and profound content. Ideas are exceptionally developed, insightful, and demonstrate exceptional originality."
    },
    "Grammar": {
        "6": "Essay is virtually flawless in terms of grammar. Sentence structure and word choice are impeccable."
    },
    "Vocabulary": {
        "6": "Essay showcases an exceptional range of vocabulary with precision and sophistication. Word choice is impeccable."
    },
    "Overall impression": {
        "6": "Essay leaves an outstanding overall impression. It is exceptionally well-organized, coherent, developed, and engaging."
    }
}


In [208]:
test_messages = []
test_messages.append({"role": "system", "content": system_message})

test_messages.append({"role": "user", "content": d_test['essay_F14'][0]})

In [209]:
response = get_completion(test_messages, model=fine_tuned_model_id)

print(d_test['essay_F14'][1])
print(response)

1.0
{
    "Organization": {
        "1": "Essay lacks clear structure and coherence. Ideas are disorganized and difficult to follow."
    },
    "Content": {
        "1": "Essay lacks relevant and meaningful content. Ideas are superficial and lack depth."
    },
    "Grammar": {
        "1": "Essay has numerous grammatical errors that impede understanding. Sentence structure and word choice are consistently incorrect."
    },
    "Vocabulary": {
        "1": "Essay has limited and inappropriate vocabulary. Word choice is often incorrect or unclear."
    },
    "Overall impression": {
        "1": "Essay leaves a negative overall impression. It lacks coherence, depth, and clarity."
    }
}


### Conclusion
The model was fine tuned and give the same results on "good" essays because these were more the essays in the train data but not enogh good on a "bad" essay, there was one example in the train data.